In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
display(tf.__version__, keras.__version__)

'2.3.0'

'2.4.0'

# 1.1 Loading 2D mnist

In [3]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
display(X_train_full.shape, X_train_full.dtype)

(60000, 28, 28)

dtype('uint8')

In [4]:
shuffle_idx = np.random.permutation(len(X_train_full))
X_train_full, y_train_full = X_train_full[shuffle_idx], y_train_full[shuffle_idx]
X_valid, X_train = X_train_full[: 5000] / 255., X_train_full[5000:] / 255.
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.

In [5]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]
np.unique(y_test)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

# 1.2 customize a layer
https://keras.io/guides/making_new_layers_and_models_via_subclassing/#putting-it-all-together-an-endtoend-example

In [6]:
# create a linear layer
class Linear(keras.layers.Layer):
    def __init__(self, units = 128):
        super(Linear, self).__init__()
        self.units = units
        
    def build(self, input_shape): # `input_shape` is a built-in variable
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value = w_init(shape = (input_shape[-1], self.units), dtype = "float32"), 
            trainable = True,
        )
        # self.w = self.add_weight(shape = (input_shape[-1], self.units), initializer = "random_normal", trainable = True)
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            initial_value = b_init(shape = (self.units,), dtype = "float32"), 
            trainable = True,
        )
        # self.b = self.add_weight(shape = (self.units,), initializer = "zeros", trainable = True)
    
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [7]:
x = tf.ones((2, 3))
linear_layer = Linear(4)
y = linear_layer(x)
print(y)

tf.Tensor(
[[-0.0630539   0.05878359  0.03976886 -0.06102928]
 [-0.0630539   0.05878359  0.03976886 -0.06102928]], shape=(2, 4), dtype=float32)


In [8]:
# layers are recursively composable
class MLPBlock(keras.layers.Layer):
    def __init__(self):
        super(MLPBlock, self).__init__()
        self.linear_1 = Linear(300)
        self.linear_2 = Linear(100)
        self.linear_3 = Linear(10)
        self.Leaky = keras.layers.LeakyReLU(alpha = 0.1)
    
    def call(self, inputs):
        self.flatten_1 = keras.layers.Flatten(input_shape = [inputs.shape[-2], inputs.shape[-1]])
        x = self.flatten_1(inputs)
        x = self.linear_1(x)
        x = self.Leaky(x)
        x = self.linear_2(x)
        x = self.Leaky(x)
        x = self.linear_3(x)
        x = tf.nn.softmax(x)
        return x

In [9]:
mlp = MLPBlock()
y = mlp(tf.ones(shape = (30, 28, 28)))
print(len(mlp.trainable_weights))

6


In [10]:
# the `add_loss()` method
class ActivityRegularizationLayer(keras.layers.Layer):
    def __init__(self, rate  = 1e-2):
        super(ActivityRegularizationLayer, self).__init__()
        self.rate = rate
    
    def call(self, inputs):
        self.add_loss(self.rate * tf.reduce_sum(inputs)) # reduce dim
        return inputs

class OuterLayer(keras.layers.Layer):
    def __init__(self):
        super(OuterLayer, self).__init__()
        self.activity_reg = ActivityRegularizationLayer(1e-2)
        
    def call(self, inputs):
        return self.activity_reg(inputs)

layer = OuterLayer()
assert len(layer.losses) == 0

_ = layer(tf.zeros(1, 1))
assert len(layer.losses) == 1

_ = layer(tf.zeros(2, 2))
len(layer.losses)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



1

In [18]:
class OuterLayerWithKernelRegularizer(keras.layers.Layer):
    def __init__(self):
        super(OuterLayerWithKernelRegularizer, self).__init__()
        self.dense = keras.layers.Dense(
            32, kernel_regularizer = tf.keras.regularizers.l1_l2(1e-3, 1e-3)
        )
    
    def call(self, inputs): 
        return self.dense(inputs)

r_layer = OuterLayerWithKernelRegularizer()
_ = r_layer(tf.zeros((1, 1)))
print(r_layer.losses)

[<tf.Tensor: shape=(), dtype=float32, numpy=0.0090072>]


In [21]:
# `add_metric()`, to track losses
class LogisticEndpoint(keras.layers.Layer):
    def __init__(self, name = None):
        super(LogisticEndpoint, self).__init__(name = name)
        self.loss_fn = keras.losses.BinaryCrossentropy(from_logits = True)
        self.accuracy_fn = keras.metrics.BinaryAccuracy()
    
    def call(self, targets, logits, sample_weights = None):
        loss = self.loss_fn(targets, logits, sample_weights)
        self.add_loss(loss)
        
        acc = self.accuracy_fn(targets, logits, sample_weights)
        self.add_metric(acc, name = "accuracy")
        prediction = tf.nn.softmax(logits)
        return prediction

In [35]:
l_layer = LogisticEndpoint()
targets = tf.ones((2, 2))
logits = tf.Tensor([[0, 1], [1, 1]], dtype = tf.float32)
_ = l_layer(targets, logits)
display(l_layer.metrics, float(l_layer.metrics[0].result()))

TypeError: __init__() missing 2 required positional arguments: 'value_index' and 'dtype'

In [40]:
tf.Tensor([4, 6], shape=(2,), dtype=np.int32)

TypeError: __init__() got an unexpected keyword argument 'shape'

# 1.3 building NN
* easy way

In [ ]:
keras.backend.clear_session()
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape = [28, 28])) # feature dimensionality
model.add(keras.layers.Dense(300, activation = "relu")) # num of neurons
model.add(keras.layers.Dense(100, activation = "relu"))
model.add(keras.layers.Dense(10, activation = "softmax"))

In [ ]:
display(model.summary())
keras.utils.plot_model(model, show_shapes = True)

## 1.2.1 inspect a single layer

In [ ]:
hidden0 = model.layers[0]
display(hidden0, hidden0.name, model.get_layer(hidden0.name) is hidden0)

In [ ]:
model.layers[1].get_weights()

In [ ]:
weights, biases = model.layers[1].get_weights()
display(weights.shape, biases.shape)